# Cross validation of the ALS method using Spark
We refer to this tutorial: http://spark.apache.org/docs/latest/ml-tuning.html#example-model-selection-via-cross-validation

The tutorial was not clear: we used SKLearn to split the dataset

# Import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


In [2]:
from pyspark.sql.functions import col

In [3]:
from helpers_cross_validation import KFoldIndexes,CrossValidation

In [4]:
import helpers

In [5]:
import random


# Models

In [6]:
class ALSModel:
    
    def __init__(self):
        pass
    
    def fit(self,data,**arg):
        self.model = ALS.train(data, **arg)
    
    def predict(self,data):
        data_for_preditions=data.map(lambda x: (x[0], x[1]))
        self.predictions = self.model.predictAll(data_for_preditions).map(lambda r: ((r[0], r[1]), r[2]))
    
    def evaluate(self,data):
        rates_and_preds = data.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(self.predictions)
        error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        return error

# Dataframe creation

In [7]:
train = helpers.load_csv()
train.head()

,UserID,MovieID,Rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


In [8]:
# First, we transform it using sqlContect
train_rdd = helpers.to_rdd(sqlContext, train)
train_rdd.take(3)

[Row(UserID=44, MovieID=1, Rating=4),
 Row(UserID=61, MovieID=1, Rating=3),
 Row(UserID=67, MovieID=1, Rating=4)]

In [9]:
train.shape[0]

1176952

# Cross validation

In [ ]:
cv=CrossValidation(train,4,True,sc)

In [ ]:
ranks = [8]
lambdas = [0.08,0.081,0.082,0.083,0.084,0.085,0.086,0.087,0.088,0.089,0.09]
numIters = [24]
nbr_models = len(ranks)*len(lambdas)*len(numIters)

bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
?ALS.train

In [ ]:
bestLambda

In [ ]:
i = 0
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    try:
        print(rank,lmbda,numIter)
        validationRmse = cv.evaluate(ALSModel(),rank=rank,lambda_=lmbda, iterations=numIter, nonnegative=True)
        validationRmse = np.mean(validationRmse)
        print("Model %i/%i: RMSE (validation) = %f" %(i+1, nbr_models, validationRmse))
        print("  Trained with rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))
        print("")
        if (validationRmse < bestValidationRmse):
#             bestModel = model
            bestValidationRmse = validationRmse
            bestRank = rank
            bestLambda = lmbda
            bestNumIter = numIter
    except:
        print("Model %i/%i failed!" %(i+1, nbr_models))
        print("  Parameters: rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))

    i += 1
    
# Evaluate the best model on the training set
print("The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
  + "and numIter = %d, and its RMSE on the training set is %f" % (bestNumIter, bestValidationRmse))

# # Evaluate the best model on the test set
# testRmse = computeRMSE(bestModel, test_for_predict_RDD, test_RDD)
# print("RMSE on the test set: %f"%(testRmse))


# Prepare data for submission

In [10]:
test = pd.read_csv('../data/sampleSubmission.csv')
test.head()

,Id,Prediction
0,r37_c1,3
1,r73_c1,3
2,r156_c1,3
3,r160_c1,3
4,r248_c1,3


In [11]:
# Prepare test for RDD
test_prep = test
test_prep['UserID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[0][1:]))
test_prep['MovieID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[1][1:]))
test_prep['Rating'] = test_prep['Prediction']
test_prep = test_prep.drop(['Prediction', 'Id'], axis=1)
test_prep.head()

,UserID,MovieID,Rating
0,37,1,3
1,73,1,3
2,156,1,3
3,160,1,3
4,248,1,3


In [12]:
# First, we transform it using sqlContect
test_sql = sqlContext.createDataFrame(test_prep)
test_rdd = test_sql.rdd
test_rdd.take(3)

[Row(UserID=37, MovieID=1, Rating=3),
 Row(UserID=73, MovieID=1, Rating=3),
 Row(UserID=156, MovieID=1, Rating=3)]

In [26]:
bestModel=ALSModel()
bestModel.fit(train_rdd,rank=8,lambda_=0.081, iterations=24, nonnegative=True)

In [27]:
bestModel.predict(test_rdd)
predictions=bestModel.predictions
# predictions = bestModel.predictAll(test_RDD_Kaggle).map(lambda r: ((r[0], r[1]), r[2]))

In [28]:
predictions.take(3)

[((4904, 864), 3.457827701935023),
 ((4904, 608), 4.273843149091485),
 ((4904, 72), 2.8437917424666415)]

In [29]:
pred_df = predictions.toDF().toPandas()

In [30]:
pred_df['UserID'] = pred_df['_1'].apply(lambda x: x['_1'])
pred_df['MovieID'] = pred_df['_1'].apply(lambda x: x['_2'])
pred_df['Rating'] = pred_df['_2']
pred_df = pred_df.drop(['_1', '_2'], axis=1)
pred_df.head()

,UserID,MovieID,Rating
0,4904,864,3.457828
1,4904,608,4.273843
2,4904,72,2.843792
3,4904,48,3.251480
4,4904,193,3.542612


In [31]:
pred_df = pred_df.sort_values(by=['MovieID', 'UserID'])
pred_df.head()

,UserID,MovieID,Rating
840969,37,1,3.212081
238375,73,1,3.092313
615219,156,1,3.712342
2964,160,1,3.294452
123927,248,1,3.485674


In [32]:
pred_df.index = range(len(pred_df))

In [33]:
test['Prediction'] = pred_df['Rating']


In [34]:
test.head()

,Id,Prediction
0,r37_c1,3.212081
1,r73_c1,3.092313
2,r156_c1,3.712342
3,r160_c1,3.294452
4,r248_c1,3.485674


In [35]:
test = test.drop(['UserID', 'MovieID', 'Rating'], axis=1)


ValueError: labels ['UserID' 'MovieID' 'Rating'] not contained in axis

In [36]:
test.to_csv('pred_pyspark_als.csv', index=False)